# Segmenting and Clustering Neighborhoods in Toronto

In this notebook, we will explore, segment, and cluster the neighborhoods in the city of Toronto.

### Scrape The Page and Convert to Dataframe

Scrape the Wikipedia Page to Get the Toronto table and transform into pandas dataframe.

In [54]:
import pandas
import requests
import numpy as np
# !pip install bs4

from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'xml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


### Drop Not Assigned Borough

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [49]:
df1 = df[df["Borough"] != "Not assigned"]
df1.reset_index(drop = True, inplace = True)
df1.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


### Cluster The Neighborhood

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [50]:
df2 = df1.groupby(['PostalCode', 'Borough'], as_index=False).agg({'Neighbourhood' : ', '.join})
df2.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Traeting Not Assigned Neighbourhood

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [51]:
df2[df2['Neighbourhood'] == 'Not assigned']

,PostalCode,Borough,Neighbourhood
93,M9A,Queen's Park,Not assigned


In [52]:
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == "Not assigned", df2['Borough'], df2['Neighbourhood'])
df2[df2['PostalCode'] == 'M9A']

,PostalCode,Borough,Neighbourhood
93,M9A,Queen's Park,Queen's Park


In [53]:
df2.shape

(103, 3)